In [1]:
import logging as l

Resized_width,Resized_height=128,128
Out_put_format='png'
Out_put_train=\
    "C:/Users/xiang/Desktop/rsna-intracranial-hemorrhage-detection-128/stage_2_train_images"
Out_put_test=\
    "C:/Users/xiang/Desktop/rsna-intracranial-hemorrhage-detection-128/stage_2_test_images"
Out_put_dir="C:/Users/xiang/Desktop/rsna-intracranial-hemorrhage-detection-128"

import glob
import joblib
import numpy as np
from PIL import Image
import pydicom
from pydicom import multival
import tqdm
import os

In [2]:
data_dir= "C:/Users/xiang/Desktop/rsna-intracranial-hemorrhage-detection"
train_dir="stage_2_train_images"
train_paths=glob.glob(f"{data_dir}/{train_dir}/*.dcm")
test_dir="stage_2_test_images"
test_paths=glob.glob(f"{data_dir}/{test_dir}/*.dcm")
print(len(train_paths),len(test_paths))

752803 121232


In [3]:
def get_first_of_dicom_as_int(x):
    if type(x)==multival.MultiValue:
        return int(x[0])
    return int(x)
def get_id(img_dicom):
    return str(img_dicom.SOPInstanceUID)

def get_metadata_from_dicom(img_dicom):
    metadata={
        "window_center":img_dicom.WindowCenter,
        "window_width":img_dicom.WindowWidth,
        "intercept":img_dicom.RescaleIntercept,
        "slope":img_dicom.RescaleSlope,
    }
    return {k:get_first_of_dicom_as_int(v) for k,v in metadata.items()}

def window_image(img,window_center,window_width,intercept,slope):
    img=img*slope+intercept
    img_min=window_center-window_width//2
    img_max=window_center+window_width//2
    img[img<img_min]=img_min
    img[img>img_max]=img_max
    return img

def resize(img,new_w,new_h):
    img=Image.fromarray(img.astype(np.int8),mode="L")
    return img.resize((new_w,new_h),resample=Image.BICUBIC)

def save_img(img_pil,subfolder,name):
    img_pil.save(f"{Out_put_dir}/{subfolder}/{name}.{Out_put_format}")

def normalize_minmax(img):
    mi,ma=img.min(),img.max()
    if mi==ma:
        return img
    return (img-mi)/(ma-mi)

def prepare_image(image_path):
    img_dicom=pydicom.read_file(image_path)
    img_id=get_id(img_dicom)
    metadata=get_metadata_from_dicom(img_dicom)
    img=window_image(img_dicom.pixel_array,**metadata)
    img=normalize_minmax(img)*255
    img_pil=resize(img,Resized_width,Resized_height)
    return img_id,img_pil
def prepare_and_save(img_path,subfolder):
#         img_id,img_pil=prepare_image(img_path)
#         save_img(img_pil,subfolder,img_id)
    try:
        l.error("loading eso")
        img_id,img_pil=prepare_image(img_path)
        save_img(img_pil,subfolder,img_id)
    except KeyboardInterrupt:
        raise
    except:
        l.error(f"error processing the image:{img_path}")

def prepare_images(imgs_path,subfolder):
    for i in tqdm.tqdm(imgs_path):
        prepare_and_save(i,subfolder)

def prepare_images_njobs(img_paths,subfolder,n_jobs=-1):
    joblib.Parallel(n_jobs=n_jobs)(joblib.delayed(prepare_and_save)(i,subfolder) for i in tqdm.tqdm(img_paths))



In [58]:
# prepare_images_njobs(test_paths,test_dir)







  0%|                                                                                       | 0/121232 [00:00<?, ?it/s]





  0%|                                                                              | 10/121232 [00:00<35:15, 57.30it/s]





  0%|                                                                              | 32/121232 [00:00<37:25, 53.97it/s]





  0%|                                                                              | 45/121232 [00:00<31:01, 65.09it/s]





  0%|                                                                              | 58/121232 [00:00<26:30, 76.19it/s]





  0%|                                                                             | 99/121232 [00:00<20:03, 100.61it/s]





  0%|                                                                            | 143/121232 [00:01<15:25, 130.82it/s]





  0%|▏                                                                           | 293/121232 [00:01<11:13, 179.52it/s]

In [59]:
# print(len(os.listdir(Out_put_test)))

121232


In [4]:
prepare_images_njobs(train_paths,train_dir)

100%|█████████████████████████████████████████████████████████████████████████| 752803/752803 [29:46<00:00, 421.32it/s]


In [8]:
print(len(os.listdir(Out_put_train)))

752802


In [9]:
prepare_images_njobs(test_paths,test_dir)

100%|█████████████████████████████████████████████████████████████████████████| 121232/121232 [04:32<00:00, 445.69it/s]


In [10]:
print(len(os.listdir(Out_put_test)))

121232


In [16]:
# error_image_indexes=[image for image in train_paths if image not in os.listdir(Out_put_train)]

KeyboardInterrupt: 